# Практическая работа №3
### Градиентный спуск

Как было рассказано на лекции, стохастический градиентый спуск сходится быстрее, чем полный, хотя и менее стабильно. В этом задании вам предлагается реализовать стохастический градиентный спуск и сравнить его с точным вычислением весов линейной модели по скорости работы и значению метрики качества.

In [46]:
import numpy as np
import warnings

np.random.seed(0)

warnings.filterwarnings('ignore')
%matplotlib inline

### Задание 0

Реализуйте класс ```LinearRegressionSGD``` c обучением и и применением линейной регрессии, построенной с помощью стохастического градиентного спуска, с заданным интерфейсом.

In [47]:
import numpy as np
from sklearn.base import BaseEstimator

class LinearRegressionSGD(BaseEstimator):
    def __init__(self, epsilon=1e-4, max_steps=100, w0=None, alpha=1e-4):
        """
        epsilon: разница для нормы изменения весов
        max_steps: максимальное количество шагов в градиентном спуске
        w0: np.array (d,) - начальные веса
        alpha: шаг обучения
        """
        self.epsilon = epsilon
        self.max_steps = max_steps
        self.w0 = w0
        self.alpha = alpha
        self.w = None
        self.w_history = []

    def fit(self, X, y):
        """
        X: np.array (l, d)
        y: np.array (l)
        ---
        output: self
        """
        l, d = X.shape
        self.w = self.w0 if self.w0 is not None else np.zeros(d)
        
        for step in range(self.max_steps):
            # Случайный выбор индекса
            idx = np.random.randint(l)
            X_i = X[idx]
            y_i = y[idx]

            # Вычисление градиента
            gradient = self.calc_gradient(X_i, y_i)

            # Обновление весов
            self.w -= self.alpha * gradient
            self.w_history.append(self.w.copy())

            # Проверка условия сходимости
            if np.linalg.norm(gradient) < self.epsilon:
                break

        return self

    def predict(self, X):
        """
        X: np.array (l, d)
        ---
        output: np.array (l)
        """
        return X @ self.w

    def calc_gradient(self, X, y):
        """
        X: np.array (d,)
        y: float
        ---
        output: np.array (d)
        """
        error = (X @ self.w) - y
        gradient = error * X
        return gradient


Проверять работу мы будем на имеющемся в sklearn наборе данных boston: в нём нужно по информации о доме предсказать его стоимость.

In [48]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Загрузка Boston Housing Data из внешнего источника
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)

# Обработка данных
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])  # Признаки
target = raw_df.values[1::2, 2]  # Целевая переменная

# Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=10)

# Создание и обучение модели
model_sgd = LinearRegressionSGD(alpha=1e-7, max_steps=1000)
model_sgd.fit(X_train, y_train)

# Предсказание и оценка качества
from sklearn.metrics import mean_squared_error
y_pred = model_sgd.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print("MSE на тестовой выборке:", mse)


MSE на тестовой выборке: 95.59030917889521


### Задание 1

Метрикой качества в нашей задаче будет MAPE - Mean Absolute Percentage Error. Реализуйте её с заданным интефейсом и вычислите
```MAPE(y_test, y_0)```, где ```y_0 = (mean(y_test), mean(y_test), ...)```

In [49]:
import numpy as np

def MAPE(y_true, y_pred):
    """
    y_true: np.array (l)
    y_pred: np.array (l)
    ---
    output: float [0, +inf)
    """
    # Вычисляем MAPE, избегая деления на ноль
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [50]:
# Создаем y_0, состоящий из среднего значения y_test
y_0 = np.full_like(y_test, fill_value=np.mean(y_test))

# Вычисляем MAPE
mape_result = MAPE(y_test, y_0)
print("MAPE(y_test, y_0):", mape_result)

MAPE(y_test, y_0): 37.41588297684096


### Задание 2

Обучите ```LinearRegressionSGD``` с базовыми параметрами на тренировочном наборе данных (```X_train```, ```y_train```), сделайте предсказание на тестовых данных ```X_test```, записав результат в переменную ```y_pred_sgd```  и вычислите ошибку MAPE.

In [51]:
# Инициализация модели с базовыми параметрами
sgd = LinearRegressionSGD()

# Обучение модели на тренировочных данных
sgd.fit(X_train, y_train)

# Предсказание на тестовых данных
y_pred_sgd = sgd.predict(X_test)

# Вычисление MAPE
mape_sgd = MAPE(y_test, y_pred_sgd)
print("MAPE(y_test, y_pred_sgd):", mape_sgd)


MAPE(y_test, y_pred_sgd): 4.826894101380071e+147


### Задание 3

Вычислите веса по точной формуле, используя ```X_train``` и ```y_train```; предскажите с их помощью целевую переменную на ```X_test```, записав результат в переменную ```y_pred_lr``` и вычислите ошибку MAPE.

In [52]:
# Добавление столбца единиц для смещения (bias) в X_train и X_test
X_train_b = np.hstack((np.ones((X_train.shape[0], 1)), X_train))  # Добавляем bias к обучающим данным
X_test_b = np.hstack((np.ones((X_test.shape[0], 1)), X_test))     # Добавляем bias к тестовым данным

# Вычисление весов по точной формуле
w_exact = np.linalg.inv(X_train_b.T @ X_train_b) @ X_train_b.T @ y_train

# Предсказание с использованием точных весов
y_pred_lr = X_test_b @ w_exact

# Вычисление MAPE
mape_lr = MAPE(y_test, y_pred_lr)
print("MAPE(y_test, y_pred_lr):", mape_lr)


MAPE(y_test, y_pred_lr): 17.39197284954152


## Бонусное задание

До этого вы релизовывали модели, в которых не было штрафа за величину весов ```w```. Как было рассказано ранее в лекциях, это может привести к неустойчивости модели и переобучению. Чтобы избежать этих эффектов, предлагается добавить к оптимизируемому функционалу L2-норму весов; таким образом, будем решать задачу гребневой регрессии, Ridge:

$$ \frac{1}{l}(Xw-y)^T(Xw-y) +\gamma||w||_2 \rightarrow \min_{w}. $$

### Задание 4
Реализуйте обучение такой модели в матричном виде (смотрите дополнительные материалы к этой неделе) с помощью стохастического градиентного спуска. Класс должен совпадать по набору реализованных функций с ```LinearRegressionVectorized```, разница будет лишь в параметре $\gamma$, отвечающем за степень регуляризации.

Так же, как и в основном задании, обучите модель с базовыми параметрами на тренировочных данных и сделайте прогноз y_pred_ridge. Выведите значение MAPE(y_test, y_pred_ridge).